In [1]:
import os
import git
from pathlib import Path
from scipy.spatial import Delaunay

ROOT_DIR =  Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
SAVE_FIGS = False

In [2]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
from reporting import *
plots_path = os.path.join(ROOT_DIR, "publication", "paper", "draft_plots")
main_df = main_df.copy()
RERUN=False
np.random.seed(0)

In [3]:
main_df = main_df.fillna("None", inplace=False)

In [ ]:
csv_path = Path(os.path.join(ROOT_DIR, 'publication', 'poster', 'CSVs', 'quick_access.csv'))

def in_hull(p, hull):
    if hasattr(hull, 'vertices') and not isinstance(hull, Delaunay):
        hull = Delaunay(hull.points)
    elif not isinstance(hull, Delaunay):
        hull = Delaunay(hull)
    return (hull.find_simplex(p) >= 0)

if RERUN or not csv_path.exists():
    x_vals = np.linspace(0, 20, 1000)
    eta_vals = 1.5 + np.zeros_like(x_vals)
    roi_beta = (eta_vals) / x_vals
    roi = 1 / roi_beta
    def line_intersects_hull(hull):
        if hull is None or hull is np.nan:
            return False
        line_points = np.column_stack((x_vals, roi))
        return np.any(in_hull(line_points, hull))

    main_df['intersect_roi'] = main_df['hull'].apply(lambda h: line_intersects_hull(h) if type(h) != str else False)
    main_df.drop('hull', axis=1).to_csv(csv_path)

main_df = pd.read_csv(csv_path).drop('Unnamed: 0', axis=1)
main_df['intersect_roi'] = (main_df['intersect_roi'] == 'True').astype(int)
main_df.head()


,group,obs_var,var_lower,var_upper,obs_kurt,kurt_lower,kurt_upper,total_samples,initial_r,initial_eta,...,dataset_type,hull,best_beta,best_1/beta,beat_all_priors,best_prior,failure_category,failure_type,which_ones,intersect_roi
0,2,1292.946700,749.107670,2108.430700,421.841550,123.974430,696.238040,45000.0,0.1,2.5,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,38.000000,0.026316,1,GenGamma,practically_pass,pass,NaN,1
1,5,662.654400,382.089750,1248.617800,628.030000,108.212240,1663.239500,117000.0,0.1,2.0,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,43.333333,0.023077,1,GenGamma,practically_pass,pass,NaN,0
2,8,256.633900,133.792040,650.845600,1363.730700,99.365600,3639.235800,189000.0,0.1,2.9,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,53.333333,0.018750,1,GenGamma,practically_pass,pass,NaN,1
3,11,121.961570,57.215298,390.192930,2698.317600,93.680930,4889.278000,495000.0,0.1,3.1,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,55.555556,0.018000,1,GenGamma,practically_pass,pass,NaN,1
4,14,48.468010,23.179575,151.679550,2654.295200,86.305040,4711.629400,1134000.0,0.1,3.4,...,remote sensing,<scipy.spatial._qhull.ConvexHull object at 0x0...,56.666667,0.017647,1,GenGamma,practically_pass,pass,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,4,241418.060000,225806.880000,257381.550000,4.450259,4.034465,4.879806,22538.0,0.5,-0.5,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x0...,1.400000,0.714286,1,GenGamma,actually_pass,pass,NaN,0
1151,5,33947.695000,31871.596000,36147.625000,3.966855,3.001096,5.099455,296776.0,0.1,8.6,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x0...,101.000000,0.009901,1,GenGamma,actually_pass,pass,NaN,1
1152,6,6416.669400,5792.756300,7090.980500,13.477583,10.768813,16.435010,2880648.0,0.1,4.2,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x0...,56.000000,0.017857,1,GenGamma,practically_pass,pass,NaN,1
1153,7,397.892600,357.501070,450.380680,17.566162,9.929181,54.616486,25362406.0,0.1,2.2,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x0...,37.000000,0.027027,1,GenGamma,practically_pass,pass,NaN,1


In [85]:
granularity = ['dataset', 'transform']
temp = main_df.copy()[(main_df['transform'] != 'learned') & (main_df['dataset'] != 'standardTesting')]
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass','intersect_roi']].mean() * 100).round(1)
summary_df

actually_pass  practically_pass  intersect_roi
dataset         transform                                                
agriVision      fourier              2.3              93.2           70.5
                wavelet             13.9              86.1           73.6
coco            wavelet             25.0              26.6           27.1
pastis          fourier             90.9               9.1          100.0
                wavelet             57.1              42.9           96.4
segmentAnything wavelet             25.0              60.2           36.1
spaceNet        fourier              0.0             100.0           50.0
                wavelet              4.7              48.4           46.9
syntheticMRI2D  wavelet              2.8              66.7           47.2
syntheticMRI3D  wavelet              6.1              53.1           57.1

In [ ]:
granularity = ['dataset_type', 'dataset', 'transform']
temp = main_df.copy()[(main_df['dataset'] != 'standardTesting')] #& (main_df['transform'] != 'learned')]
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass','intersect_roi']].mean() * 100).round(1).sort_values('dataset_type')
summary_df

,actually_pass,practically_pass,intersect_roi
dataset_type,,,
medical,4.1,61.2,51.2
natural,15.3,48.2,33.7
remote sensing,18.6,63.7,67.4


In [82]:
save_path = Path(os.path.join(ROOT_DIR, 'publication', 'poster', 'CSVs', 'summary_table.csv'))
summary_df.to_csv(save_path) #.drop('learned')

In [86]:
main_df[['actually_pass','practically_pass','intersect_roi']].mean(numeric_only=True)*100

actually_pass       14.545455
practically_pass    54.199134
intersect_roi       50.476190
dtype: float64